In [1]:
import pandas as pd
import numpy as np
import dimod
from itertools import combinations

%load_ext autoreload
%autoreload 1

%aimport modules
from modules import qubo_from_data
from modules import process_qubo_df
from modules import reduce_higher_order_terms
from modules import qubo_matrix_from_df
from modules import print_qubo_matrix
from modules import read_spot_file

## QUBO formulation

### Data description

Our task consists in optimizing the routine of a satellite that needs to take several photos. The satellite under consideration has three different cameras (1,2,3) and can take mono-photo, choosing one among the three cameras, or stereo-photo, with cameras 1 and 3.

The satellite's routine is scheduled by a .spot file that contains both photo requests and constraints on those. At the beginning of the requests, there is a single line stating the number of requests, the same for the constraints.

A photo request consists in:
- the id of the photo
- the value of the picture
- a list of different cameras with which the photo can be taken
- (after the camera id there is the memory used, not of our interest yet)

A constraint consists in:
- the number of requests affected by the constraint
- a list of camera combinations not allowed

Our goal is to maximize the value of each .spot schedule without breaking any constraints.

### Read spot file

In [2]:
# read spot file
data_dir  = 'data/'
file_name = '20.spot'
photo_req_df, constraints_df = read_spot_file(data_dir, file_name)

In [3]:
# visualizing photo requests
photo_req_df.head()

,id,value,mono,options
0,0,2,False,[13]
1,1,1,True,"[1, 2, 3]"
2,2,1,True,"[1, 2, 3]"
3,3,2,False,[13]
4,4,2,False,[13]


In [4]:
# visualizing constraints
constraints_df.head()

,ids,restrictions
0,"[2, 1]","[[3, 3], [2, 2], [1, 1]]"
1,"[2, 10]","[[3, 13], [1, 13]]"
2,"[4, 6]","[[13, 13]]"
3,"[1, 10]","[[3, 13], [1, 13]]"
4,"[3, 12]","[[13, 13]]"


### Encoding
QUBO stays for Quadratic Uncostrained _Binary_ Optimization. First of all, we should translate our problem in terms of binary variables. We will consider two possibilities:

__standard__
- _mono_: We can take (1) or not take (0) the photo with each of the three cameras. Three binary variables are needed:
$$
(x_{i0},\, x_{i1},\, x_{i2})
$$
- _stereo_: The stereo photo can only be taken with the cameras $1$ and $3$, so we have only one binary variable:
$$
x_i
$$

Note that, in the case of a mono photo, we have a total of 8 different instantiations of the binary variables when only 3 are feasible ($001,\, 010,\, 100$). We can think about more efficient encodings.

__dense__:
- _mono_: Just two variables could be sufficient:

<center>

|          | $x_{i0}$ | $x_{i1}$ |
|----------|----------|----------|
| no photo | 0        | 0        |
| camera 1 | 0        | 1        |
| camera 2 | 1        | 0        |
| camera 3 | 1        | 1        |

</center>

- _stereo_: As in the previous encoding we consider one variable
$$
x_i
$$

In the latter case we use one less variable with respect to the former, it seems convenient. Despite that, with the _dense_ encoding the costraint formulation requires an auxiliary variable so it is important to carefully understand if there is an actual advantage.

In [5]:
# select encoding (standard, dense)
encoding = 'dense'
penalty_coeff = 1.1

# get qubo df
qubo_df = qubo_from_data(photo_req_df, constraints_df, encoding, penalty_coeff)

In [6]:
# each row is a term in the qubo
# the indexes column has the following pattern [[id, camera], ...]
# each tuple [id, camera] is a variable in the qubo formulation
# when more than one tuple is present, the term is a higher order term (>1)
qubo_df.head()

,rank,coeff,indexes
0,1,-2.0,"[[0, 13]]"
1,2,1.0,"[[1, 0], [1, 1]]"
2,1,-1.0,"[[1, 1]]"
3,1,-1.0,"[[1, 0]]"
4,2,1.0,"[[1, 0], [1, 1]]"


### Preprocessing

The _tuple_ variables indexing is convenient to link the variable to its actual meaning but not efficient in terms of handling, we need to convert in a unique way the _tuple_ indexing to a standard integer indexing. We also have to group by same terms in the dataframe summing the coefficients.

The _tuple_ to _integer_ conversion is stored in the `key_to_qubo_dict`.

In [7]:
qubo_df_, key_to_qubo_dict = process_qubo_df(qubo_df)

In [8]:
qubo_df_.head()

,rank,coeff,variables
0,1,-2.0,[0]
1,2,1.0,"[14, 15]"
2,1,-1.0,[15]
3,1,-1.0,[14]
4,2,1.0,"[16, 17]"


In [9]:
print(key_to_qubo_dict)

{'0_13': 0, '10_13': 1, '11_0': 2, '11_1': 3, '12_13': 4, '13_13': 5, '14_13': 6, '15_0': 7, '15_1': 8, '16_0': 9, '16_1': 10, '17_13': 11, '18_13': 12, '19_13': 13, '1_0': 14, '1_1': 15, '2_0': 16, '2_1': 17, '3_13': 18, '4_13': 19, '5_0': 20, '5_1': 21, '6_13': 22, '7_0': 23, '7_1': 24, '8_0': 25, '8_1': 26, '9_13': 27}


### Reducing higher order terms

A QUBO formulations requires only up to quadratic terms, in our use case we have different scenarios in which we have to deal with higher order terms (order>2).
- __standard encoding__ : three photo costraints generate cubic terms
- __dense encoding__: already two photo costraints generate quartic terms, up to order six in the case of three photo costraints.

Two main method have been explored to reduce higher order terms: _boros_ [[1]] and _ishikawa_ [[2]]. Also a mixed method involving these two have been explored _mix_.

[1]: https://www.sciencedirect.com/science/article/pii/S0166218X01003419?via%3Dihub
[2]: https://ieeexplore.ieee.org/document/5444874

In [10]:
# set method (boros, ishikawa, mix)
method = 'boros'

# reduce higher order terms
qubo_df_, key_to_qubo_dict = reduce_higher_order_terms(qubo_df_, key_to_qubo_dict, method)

### Solving QUBO formulations

Beyond formulating a QUBO in the testing phase we were interested in checking the solutions of our QUBO formulations to see wether they were correct. Here we use both an exact solver (only feasible for smaller files) and a simulated annealing one.

In [13]:
# get qubo matrix from qubo_df_
qubo = qubo_matrix_from_df(qubo_df_)

#### Exact solver

In [ ]:
# encode the dataframe into a qubo object
bqm = dimod.BQM.from_qubo(qubo)
# exact solver
sampler_exact = dimod.ExactSolver()
sampleset = sampler_exact.sample(bqm)

In [ ]:
# save results in a dataframe
results_df = sampleset.to_pandas_dataframe()
# recover original indexes
qubo_to_key_dict = {v: k for k, v in key_to_qubo_dict.items()}
results_df.columns = [qubo_to_key_dict[q] for q in results_df.columns[:-2]]+['energy', 'num_occurrences']
# sort by energy
results_df = results_df.sort_values(by=['energy'], ascending=True)

In [ ]:
results_df.head()

#### Simulated annealing sampler

In [14]:
# encode the dataframe into a qubo object
bqm = dimod.BQM.from_qubo(qubo)
# SA solver
sampler_exact = dimod.SimulatedAnnealingSampler()
sampleset = sampler_exact.sample(bqm, num_reads=100, num_sweeps=500)

In [15]:
# save results in a dataframe
results_df = sampleset.to_pandas_dataframe()
# recover original indexes
qubo_to_key_dict = {v: k for k, v in key_to_qubo_dict.items()}
results_df.columns = [qubo_to_key_dict[q] for q in results_df.columns[:-2]]+['energy', 'num_occurrences']
# sort by energy
results_df = results_df.sort_values(by=['energy'], ascending=True)

In [16]:
results_df.head()

,0_13,10_13,11_0,11_1,12_13,13_13,14_13,15_0,15_1,16_0,...,11_0_11_1,11_0_16_0,11_1_16_0,7_0_16_0,7_1_16_0,8_0_16_0,8_1_16_0,16_0_16_1,energy,num_occurrences
55,1,0,0,1,1,1,1,0,1,0,...,0,0,0,0,0,0,0,0,-24.0,1
61,1,0,1,1,1,1,0,0,1,0,...,1,0,0,0,0,0,0,0,-24.0,1
20,1,0,1,0,1,1,0,0,1,0,...,0,0,0,0,0,0,0,0,-23.0,1
96,1,0,0,1,1,1,1,0,1,0,...,0,0,0,0,0,0,0,0,-23.0,1
52,1,0,0,0,1,1,0,0,1,0,...,0,0,0,0,0,0,0,0,-23.0,1
